In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import sys
sys.path.append('/home/simon/Research/lib/caffe.py3-2/python/')
import caffe
caffe.set_device(0)
caffe.set_mode_gpu()
import Caffe
import Resize
import Pad
import PIL
import Classification
import glob

import matplotlib
%matplotlib inline  
import matplotlib.pyplot as plt
import Dataset
import Evaluation
import ImageReader
import Crop
import MulticlassSVM
import Classificator
import sklearn
import Lambda

In [ ]:
!pwd

In [ ]:
basedir = '/home/simon/Datasets/CUB_200_2011/'
d = Dataset.Dataset()
d.read_from_file('%s/imagelist_absolute.txt'%basedir, 'imagepaths','string')
d.read_from_file('%s/labels.txt'%basedir,'labels','int')
d.read_from_file('%s/tr_ID.txt'%basedir,'split_assignments','int')

In [ ]:
c = Classification.Classification()
c.add_algorithm(ImageReader.ImageReader())
c.add_algorithm(Resize.Resize((512,512), mode='stretch'))
c.add_algorithm(Crop.Crop((448,448),'center'))
#c.add_algorithm(Pad.Pad(470,470))
#modeldir = '/home/simon/Data/caffe/vgg16/cub200_bilinear/'
#modeldir = '/home/simon/Research/generic/results/2016-07-01_cnn_imagenet_training/alexnet-visualization-nofc/alexnet_vis_nofc_imagenet/'
modeldir = '/home/simon/Data/caffe/caffe/bilinear'
c.add_algorithm(Caffe.Caffe(modeldir + '/deploy.prototxt',
                           modeldir + '/model',
                           mean = np.float32([104.0, 116.0, 122.0]),
                           outblob = 'bilinear_l2',
                           batchsize = 10))
#c.add_algorithm(Lambda.Lambda(lambda x: x.sum(axis=(1,2))))
c.add_algorithm(Classificator.Classificator(sklearn.svm.LinearSVC(C=1)))
#c.add_algorithm(Classificator.Classificator(sklearn.svm.SVC(C=1.0, kernel='poly', degree=2)))

In [ ]:
ev = Evaluation.Evaluation()
ev.fixed_split_eval(d, c)

In [ ]:
test_images = [im for im,split in zip(d.imagepaths,d.split_assignments) if split==0]
test_labels = [label for label,split in zip(d.labels,d.split_assignments) if split==0]

In [ ]:
predictions = c.compute_all(test_images)

In [ ]:
np.savetxt('/home/simon/Research/finegrained/results/2016-09-02_single_pose_dataset/cub200_2011/vgg16_bilinear_pred/results.txt',np.vstack(predictions).astype(int),fmt='%i')